In [ ]:
!pip install transformers
!pip install -q -U watermark
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import XLNetForSequenceClassification, Trainer, TrainingArguments

# Load dataset
df = pd.read_csv('/content/combined_data.csv')

# Ensure dataset columns are correctly named
df.columns = ['label', 'text']

# Split dataset into training and testing
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

# Initialize XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Tokenize the data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)


In [ ]:
train_encodings = train_encodings.data if hasattr(train_encodings, 'data') else train_encodings
test_encodings = test_encodings.data if hasattr(test_encodings,'data') else test_encodings
print(train_encodings.keys())

In [ ]:
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)
train_dataset = EmailDataset(train_encodings, train_labels.tolist())
test_dataset = EmailDataset(test_encodings, test_labels.tolist())

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")


In [ ]:
# Load XLNet model for classification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./new_results1',
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    no_cuda = not torch.cuda.is_available(),
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


In [ ]:

# Define evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to predicted labels
    predictions = np.argmax(logits, axis=1)  # Use numpy here as logits are numpy arrays
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

# Add compute_metrics to the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # Include the metrics function
)

# Run evaluation
results = trainer.evaluate()
print("Evaluation results:", results)